In [6]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F
import os
import pandas as pd

In [2]:
if 'spark' in locals() and spark!=None:
    spark.stop()


spark=None

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/20 15:54:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/20 15:54:35 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [3]:
chd = spark.read \
    .option("delimiter", "\t") \
    .csv("chd_out.csv.gz/*.csv.gz", header=True)

In [8]:
keys_to_extract = [#NONE CAN BE SUBSTRINGS OF THE OTHERS
    "K562__ref", "HepG2__ref", "SKNSH__ref", "K562__alt", "HepG2__alt", "SKNSH__alt",
    "K562__skew", "HepG2__skew", "SKNSH__skew",
]

# Apply the regexp_extract function to the DataFrame to create new columns for each key.
# The expression '([^;]*)' captures any sequence of characters that are not a semicolon,
# which is assumed to be the delimiter for the key-value pairs in the 'INFO' column.

for key in keys_to_extract:

    #df = df.withColumn(key, regexp_extract(col("INFO"), "{}=([^;]+);?".format(key), 1))
    #when we find something put it, whne we don't put None
    chd = chd.withColumn(key, 
                       F.when(
                           F.regexp_extract(F.col("INFO"), "{}=([^;]+);?".format(key), 1) != "",
                           F.regexp_extract(F.col("INFO"), "{}=([^;]+);?".format(key), 1)).otherwise(None))

In [10]:
chd=chd.drop("Info")

rsID, which we joined on, is NOT enough to uniquiely identify a SNP! We will use the nucleotides, too. 

In [37]:
ref=chd.filter(F.col("which").startswith("Ref"))
alt=chd.filter(F.col("which").startswith("Non"))

In [48]:
ref_correct_nt=ref.filter(ref.allele==ref.REF)

In [49]:
alt_correct_nt=alt.filter(ref.allele==ref.ALT)

In [53]:
no_prediction=chd.filter(F.col("REF").isNull())

The number of alleles in the original spreadsheet is 14524

In [59]:
alt_correct_nt.count()+ref_correct_nt.count()

11804

In [22]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(alt_correct_nt.limit(5).toPandas())

,twist_seq,seq,which,allele,CHROM,POS,ID,REF,ALT,QUAL,FILTER,K562__ref,HepG2__ref,SKNSH__ref,K562__alt,HepG2__alt,SKNSH__alt,K562__skew,HepG2__skew,SKNSH__skew
0,ACTGGCCGCTTGACGTTTGTTTCAAAGAATGACATTCCAATATTCT...,rs10053901,Non-Ref,A,chr5,23327847,rs10053901,T,A,.,.,0.24528538,0.17082982,0.28918344,0.24863826,0.16324805,0.2590601,0.0033528863,-0.007581764,-0.03012329
1,ACTGGCCGCTTGACGTTTGTTTCAAAGAATGACATTCCAATATTCT...,rs10053901,Non-Ref,G,chr5,23327847,rs10053901,T,G,.,.,0.24528538,0.17082982,0.28918344,0.24810076,0.18790716,0.31679872,0.002815386,0.01707734,0.027615324
2,ACTGGCCGCTTGACGGTTGGAAGTGAGCAGTCATCTAAGGTCTACG...,rs10065718,Non-Ref,T,chr5,104701020,rs10065718,A,T,.,.,0.6069474,0.7525695,0.51740646,0.6839925,0.8182982,0.6036991,0.07704512,0.06572859,0.086292565
3,ACTGGCCGCTTGACGGATTAGGTCATAGTAAGGCTACTGTCTTTTA...,rs10428319,Non-Ref,C,chr4,77922471,rs10428319,G,C,.,.,0.045638386,0.037590988,-0.28576174,0.03520652,0.0084324125,-0.30588984,-0.010431869,-0.02915858,-0.020128114
4,ACTGGCCGCTTGACGAAAATTCTCTTTTTTTGTTGTGTCTCTGCCC...,rs10897362,Non-Ref,A,chr11,63111386,rs10897362,T,A,.,.,-0.06181907,-0.17066376,0.06886074,-0.08515031,-0.16880952,0.08658069,-0.02333124,0.0018542452,0.017719954


In [61]:
final=alt_correct_nt.union(ref_correct_nt)
final.coalesce(1).write.csv("predictions.tsv", sep="\t", header=True, mode="overwrite")